# Lab 4, Exercise 4

In [ ]:
import numpy as np
import pandas as pd

## Load data 


In [ ]:
# Load the data in the following two CSVs:
# data/exercise4/lab4_normal_data.csv
# data/exercise4/lab4_malicious_data.csv
# The first consists completely of normal data, while the second consists completely of malicious data
# Note: Both sets of data contain the same features used in Exercise 1; the data has already been preprocessed
# (i.e., you can keep all the features and there are no labels in the CSVs)

# CODE HERE

In [ ]:
# Create 15 datasets, where the ith dataset consists of:
# - all normal data
# - only the ith malicious datapoint

# CODE HERE


## Anomaly detection

In [ ]:
# For each dataset, run isolation forests
#
# Use the following evaluation metric:
# - rank the anomalousness of each datapoint using the isolation forest
# - record the list index of each attack datapoint when sorting from most to least unusual
#     - e.g., if the attack datapoint is at index 0 in the list, we want to record the value 0
#
# Note: don't worry about ties in ranking
# Hint: What is the difference between isolation forest's 'decision_function' and 'predict' methods? 

# CODE HERE

## Questions:
1) Why is there no separate training and test set?

2) What is the metric measuring?  What would be a perfect score?  Bonus: What is the expected performance of an outlier detector that assigns a random score to each datapoint?

3) How well does the isolation forest perform compared to a perfect score? Bonus: How well does the isolation forest perform compared to a random detector?

4) What are some issues that would prevent this model from being practically deployed?

5) What might happen if we inject five attack datapoints at a time?  What might happen if we inject 100 attack datapoints at a time?

6) What is the effect of the parameters max_features and max_samples?  What other parameters could you adjust to change performance?

Optional: What are some alternative anomaly detection models one could use instead of an isolation forest? Bonus: Try one of these alternatives and compare performance.